In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
df = pd.read_csv("Updated_GrammarandProductReviews.csv")

In [3]:
df.head(5)

,Unnamed: 0,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,...,reviews_doRecommend,reviews_id,reviews_numHelpful,reviews_rating,reviews_sourceURLs,reviews_text,reviews_title,reviews_username,upc,Review
0,0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,...,NaN,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Joshua,6.02537E+11,awesome love album good hip hop side current p...
1,1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,...,NaN,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,Dorothy W,73416000391,good good flavor review collect part promotion
2,2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,...,NaN,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,Dorothy W,73416000391,good good flavor
3,3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,...,False,113026909.0,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,Rebecca,67981934427,disappoint read review look buy one couple lub...
4,4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,...,False,171267657.0,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,Walker557,67981934427,irritation husband buy gel us gel caused irrit...


In [4]:
df.columns

Index(['Unnamed: 0', 'id', 'brand', 'categories', 'dateAdded', 'dateUpdated',
       'ean', 'keys', 'manufacturer', 'manufacturerNumber', 'name',
       'reviews_date', 'reviews_dateAdded', 'reviews_dateSeen',
       'reviews_didPurchase', 'reviews_doRecommend', 'reviews_id',
       'reviews_numHelpful', 'reviews_rating', 'reviews_sourceURLs',
       'reviews_text', 'reviews_title', 'reviews_username', 'upc', 'Review'],
      dtype='object')

In [5]:
df = df[["reviews_username","ean","reviews_rating","name"]]

In [6]:
# Dropping all missing values
df = df.dropna()

In [7]:
# Renaming the columns
df.rename(columns={'reviews_username' : 'userID','ean':'prod_ID','reviews_rating':'rating','name':'prod_name'}, inplace=True)

In [8]:
print("Total data ")
print("-"*50)
print("\n Total no of ratings :",df.shape[0])
print("Total No of Users   :", len(np.unique(df.userID)))
print("Total No of products  :", len(np.unique(df.prod_ID)))

Total data 
--------------------------------------------------

 Total no of ratings : 3843
Total No of Users   : 3617
Total No of products  : 77


In [9]:
df.isnull().any().any()

False

In [10]:
df['prod_ID'] = df['prod_ID'].apply(lambda x: x.replace(',',''))

In [11]:
df['prod_ID'] = df['prod_ID'].apply(lambda x: str(int(float(x))))

In [12]:
counts1=df['userID'].value_counts() 
counts=df['prod_ID'].value_counts()

In [13]:
len(counts1[counts1>=2].index)

114

In [14]:
len(counts[counts>=2].index)

60

In [15]:
df1=df[df['userID'].isin(counts1[counts1 >=2].index)]
df1.shape

(340, 4)

In [16]:
df1=df1[df1['prod_ID'].isin(counts[counts >=1].index)]
df1.shape

(340, 4)

In [17]:
df1.head()

,userID,prod_ID,rating,prod_name
0,Joshua,602537000000,5,Pink Friday: Roman Reloaded Re-Up (w/dvd)
1,Dorothy W,73416000391,5,Lundberg Organic Cinnamon Toast Rice Cakes
2,Dorothy W,73416000391,5,Lundberg Organic Cinnamon Toast Rice Cakes
3,Rebecca,67981934427,1,K-Y Love Sensuality Pleasure Gel
10,jds1992,67981934427,1,K-Y Love Sensuality Pleasure Gel


In [18]:
ratings_sum = pd.DataFrame(df1.groupby(['prod_ID'])['rating'].sum()).rename(columns = {'rating': 'ratings_sum'})
top10 = ratings_sum.sort_values('ratings_sum', ascending = False).head(10)
top10

,ratings_sum
prod_ID,
78787107731,315
25192091490,199
75609007408,134
19800001285,129
3760024193900370291719053812728594432,80
883929000000,69
857249000000,46
381519000000,45
78000148169,33


In [19]:
top10_popular=top10.merge(df1,left_index = True, right_on = 'prod_ID').drop_duplicates(
    ['prod_ID', 'prod_name'])[['prod_ID', 'prod_name','ratings_sum']]

In [20]:
print ('Top 10 Popular Products by sum user ratings\n')
top10_popular

Top 10 Popular Products by sum user ratings



,prod_ID,prod_name,ratings_sum
3813,78787107731,"Pendaflex174 Divide It Up File Folder, Multi S...",315
5703,25192091490,Red (special Edition) (dvdvideo),199
56,75609007408,Olay Regenerist Deep Hydration Regenerating Cream,134
720,19800001285,Windex Original Glass Cleaner Refill 67.6oz (2...,129
6358,3760024193900370291719053812728594432,"Hormel Chili, No Beans",80
5504,883929000000,100:Complete First Season (blu-Ray),69
4581,857249000000,Stander Pt Bedcane - Bed Handle,46
1096,381519000000,"Aussie Aussome Volume Shampoo, 13.5 Oz",45
3730,78000148169,Diet Canada Dry Ginger Ale - 12pk/12 Fl Oz Cans,33
1257,51700881093,"Finish Quantum Dishwasher Detergent, Lemon Spa...",30


In [23]:
ratingsd=df1.pivot_table(index='prod_ID',columns= 'userID',values='rating',aggfunc='mean').fillna(0)
ratingsd.head()

userID,143st,AGMT,Alex,An anonymous customer,Andi,Angel,Anne,Annie,Anonymous,April,...,catcrazygal,cindy95,eddie888,gamerguy,jds1992,mg6020,pimc-utta,regal76,spanky,tw350z
prod_ID,,,,,,,,,,,,,,,,,,,,,
13317014165,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14800646177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16000424456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19800001285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
22400264493,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df1["prod_ID"].value_counts()

78787107731                              78
25192091490                              42
19800001285                              30
75609007408                              29
3760024193900370291719053812728594432    20
883929000000                             15
381519000000                             10
857249000000                             10
78000148169                               7
79656007800                               7
51700881093                               6
858293000000                              5
54402250075                               5
79400920706                               5
67981934427                               4
810709000000                              4
22400264493                               4
13317014165                               4
381371000000                              4
488940475960047991521280                  3
16000424456                               3
51651093866                               3
652729000000                    